# Aggressivity analysis
## Who are the driver of the polarization?

## Setup

In [1]:
# Notebook config
%config Completer.use_jedi = False

In [2]:
# Built-in
import os
from IPython.display import display

# Third parties
import numpy as np
import pandas as pd
import nltk
from nltk import tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import scipy
import statsmodels.api as sm
import statsmodels.formula.api as smf


In [3]:
# Initialization needed for some modules

# tqdm for pandas
tqdm.pandas()

# NLTK configuration
nltk.download('vader_lexicon')
nltk.download('stopwords')
sia = SentimentIntensityAnalyzer()

# TokenSpace initialization
tokenSpace = tokenize.WhitespaceTokenizer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/olivier/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/olivier/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# Configuration
DATA_PATH = "data"
PKL_PATH = os.path.join(DATA_PATH, "pkl")
CSV_PATH = os.path.join(DATA_PATH, "csv")
RESOURCES_PATH = os.path.join(DATA_PATH, "resources")

In [5]:
# Utils functions

def get_sentiment(row: pd.Series) -> pd.Series:
    """
    Compute the sentiment score of a given row
    """   
    
    row['NLTK_score'] = sia.polarity_scores(row['quotation'])
    return row

def counter(text, columnText, quantity, label):
    allWords = ' '.join([text for text in text[columnText].astype('str')])
    tokenPhrase = tokenSpace.tokenize(allWords)
    frequency = nltk.FreqDist(tokenPhrase) 
    dfFrequency = pd.DataFrame({"Word": list(frequency.keys()), "Frequency": list(frequency.values())}) 
    
    dfFrequency = dfFrequency.nlargest(columns = "Frequency", n = quantity)
    plt.figure(figsize=(15,3))
    ax = sns.barplot(data = dfFrequency, x = "Word", y = "Frequency", palette="deep")
    ax.set(ylabel = "Count")
    plt.xticks(rotation='horizontal')
    plt.title(f"Most common words for {label}")
    plt.show()

In [6]:
# Load df
df = pd.read_pickle(os.path.join(PKL_PATH, "final_subset.pkl"))

### For windows users :
# from pickle5 import pickle
# with open("data/pkl/final_subset.pkl", "rb") as fh:
#   df = pickle.load(fh)

In [7]:
display(df.info())
df.sample(2)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105929 entries, 0 to 6361
Data columns (total 31 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   quoteID               105929 non-null  object 
 1   quotation             105929 non-null  object 
 2   speaker               105929 non-null  object 
 3   qids                  105929 non-null  object 
 4   date                  105929 non-null  object 
 5   numOccurrences        105929 non-null  float64
 6   probas                105929 non-null  object 
 7   urls                  105929 non-null  object 
 8   phase                 105929 non-null  object 
 9   subset                105929 non-null  bool   
 10  id                    82439 non-null   object 
 11  givenName             105929 non-null  object 
 12  familyName            105929 non-null  object 
 13  unaccentedGivenName   105929 non-null  object 
 14  unaccentedFamilyName  105929 non-null  object 
 15  bi

None

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase,subset,...,honorificPrefix,honorificSuffix,position,stateName,parties,NLTK score,neg,neu,pos,compound
12540,2016-06-16-001067,it's pretty clear democrats aren't going to le...,john cornyn,['Q719568'],2016-06-16 12:40:47,1.0,"[['John Cornyn', '0.517'], ['None', '0.4579'],...",['http://wgno.com/2016/06/16/ive-had-enough-se...,E,True,...,NaN,NaN,Senator,TX,Republican,"{'neg': 0.0, 'neu': 0.721, 'pos': 0.279, 'comp...",0.0,0.721,0.279,0.7003
9439,2015-04-07-054358,"on syria and ukraine, the person he's closest ...",mike rogers,"['Q1933467', 'Q970515']",2015-04-07 04:01:00,1.0,"[['Mike Rogers', '0.3889'], ['Elliott Abrams',...",['http://www.usnews.com/news/blogs/run-2016/20...,E,True,...,NaN,NaN,Representative,MI,Republican,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,1.000,0.000,0.0000


## 1. Aggressivity analysis

Which politicians are the least or the most aggressive.

To see that, we will simply check the most negative scores.

In [28]:
most_agg = df.groupby(["speaker", "parties"]).agg({
    "compound": "mean",
    "speaker": "size", 
}) \
.rename({"speaker": "quotes_count"}, axis=1) \
.sort_values("compound")

most_agg.head(5)

,,compound,quotes_count
speaker,parties,,
robert hayes,Republican,-0.8910,1
david hobson,Republican,-0.7841,1
ron lewis,Republican,-0.7783,1
john shadegg,Republican,-0.7351,1
heath shuler,Democrat,-0.6249,1


As we can see, the extremes are such because they only have one quote to their names. So we will only consider those with at least 100 quotes.

In [20]:
most_agg = most_agg[most_agg["quotes_count"] >= 100]
# Only keep those really negative
threshold = -0.05
most_agg = most_agg[most_agg["compound"] <= threshold].sort_values("compound")
most_agg

,,compound,quotes_count
speaker,parties,,
barbara lee,Democrat,-0.211459,182
tulsi gabbard,Democrat,-0.139305,332
joe walsh,Republican,-0.096052,105
elijah cummings,Democrat,-0.095461,936
maxine waters,Democrat,-0.090463,480
bennie thompson,Democrat,-0.087981,129
pramila jayapal,Democrat,-0.073166,270
claudia tenney,Republican,-0.070770,117
john yarmuth,Democrat,-0.070638,226


In [21]:
g = pd.DataFrame(most_agg.groupby("parties").size()).rename({0: "count"}, axis=1)
g["proportion"] = g["count"] / g["count"].sum()
g
# Less republicans

,count,proportion
parties,,
Democrat,14,0.823529
Republican,3,0.176471


## 1.1 EDA of most agressive speakers

It seems that the following speakers are specifically negative and thus, polarizing when they are mentioning people of the other political camp:
- barbara lee (Democrat)
- tulsi gabbard (Democrat)
- joe walsh (Republican)

In [30]:
speakers = ["barbara lee", "tulsi gabbard", "joe walsh"]

for speaker in speakers:
    print("Samples from", speaker)
    quotes = df[df["speaker"] == speaker].sample(3)[["quoteID", "compound", "quotation"]].values
    for q in quotes:
        print(f"QID: {q[0]}, Score: {q[1]} \n \"{q[-1]}\"\n")
    print("-" * 50)

Samples from barbara lee
QID: 2017-03-10-024778, Score: -0.8176 
 "for more than fifteen years, the u.s. has been engaged in an ever-expanding war in the middle east. president trump's deployment of combat troops in syria is the latest front in this endless war."

QID: 2018-03-13-150933, Score: -0.8757 
 "we need a seasoned expert leading the state department, not a rubber stamp for president trump's military-first foreign policy agenda. additionally, i am appalled by the nomination of gina haspel as cia director. her role as an architect of the bush administration's illegal torture program should disqualify her from serving in this critical position. our cia director should respect human rights and international law."

QID: 2017-06-29-069197, Score: -0.1796 
 "i've been working on this for years and years and years. i'm just really pleased that republicans and democrats today really understood what i've been saying and i've been explaining for the last 16 years, and that is, this reso

**Barbara Lee**
Interesting quotes:
- QID: 2015-09-16-064526 Score: -0.743   
 "it's past time that republicans stop governing by crisis,"
- QID: 2017-12-21-143552 Score: -0.3252   
 "you will not have a united states government as we know it if donald trump has his way."
- QID: 2017-07-21-019041 Score: -0.8452   
 "congress has no business interfering in women's personal health decisions. my republican colleagues need to drop this dangerous crusade and stop trying to turn back the clock on women."

 
Example of our algorithm's weaknesses:
- QID: 2017-06-29-069197 Score: -0.1796   
 "i've been working on this for years and years and years. i'm just really pleased that republicans and democrats today really understood what i've been saying and i've been explaining for the last 16 years, and that is, this resolution is a blank check for perpetual war,"
    - not against reps
- QID: 2020-04-16-025948 Score: 0.8074   
 "instead of giving relief to americans who are struggling to make ends meet, senate republicans snuck in tax breaks and corporate giveaways for their wealthy friends,"
    - should be negative
    
=> many many quotes about trump


**Tulsi Gabbard** quotes:
- QID: 2019-09-25-056870, Score: -0.3182   
 "it's important that donald trump is defeated,"
- QID: 2020-01-10-062819, Score: -0.9734  
 "president trump has committed an illegal and unconstitutional act of war, pushing our nation headlong into a war with iran without any authorization from congress -- a war so devastating and costly it would make our wars in iraq and afghanistan look like a picnic,"
 
Problems:
- QID: 2019-05-21-110771, Score: -0.5267  
 "we can and must do so by recognizing that the effects of climate change are threatening people in communities all across the country, whether you're in a republican state or a democratic state. in order to bring about the kind of big change that we need to see, we have to come together and unite toward making the big investments that we need to make."
    - should be the contrary of polarizing
- QID: 2019-04-30-091725, Score: -0.4927  
 "the most attacks i get are not from republicans,"
    - actually in favor of republicans

if democrats are almost only mentioning Trump, what's happening with republicans?

**Joe Walsh**:
- QID: 2019-08-24-040425, Score: -0.3818  
 "the truth: as practiced by most muslims, islam is not a religion. these muslims are at war w us. barack obama, a muslim, is on their side,"
- QID: 2018-09-04-100175, Score: -0.2023  
 "this whole process is a joke. all the democrats demanding to review all these kavanaugh documents are the very same democrats who announced months ago they were going to oppose kavanaugh no matter what. so why do they need to review anything?"
- QID: 2017-11-30-108399, Score: -0.4939  
 "the next time you hear a democrat talk about protecting illegals or defending sanctuary cities, remind them that it's because of an illegal and sanctuary cities that kate steinle is no longer alive today."

Problems:
..


Almost all quotes speak about trump or its impeachment in a way or another. We should really investigate the impact of some events on the cleavage.

Before that, we will investigate the aggressivity's evolution through the years. 

## 1.2 Aggressivity by year

In [13]:
# create new "year" column in df
df["date"] = pd.to_datetime(df["date"])
df["year"] = df["date"].dt.year

In [14]:
# Sanity check
df.sample(1)

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase,subset,...,honorificSuffix,position,stateName,parties,NLTK score,neg,neu,pos,compound,year
2665,2019-10-22-057060,"just the facts, baby. if we tell that story wi...",hakeem jeffries,['Q5640425'],2019-10-22 13:11:08,3.0,"[['Hakeem Jeffries', '0.9085'], ['None', '0.09...",['https://thefederalist.com/2019/10/22/the-clo...,E,True,...,NaN,Representative,NY,Democrat,"{'neg': 0.048, 'neu': 0.845, 'pos': 0.107, 'co...",0.048,0.845,0.107,0.4404,2019


In [31]:
most_agg = df.groupby(["year", "speaker", "parties"]).agg({
    "compound": "mean",
    "speaker": "size", 
}) \
.rename({"speaker": "quotes_count"}, axis=1) \
.sort_values("compound")

most_agg = most_agg[most_agg["quotes_count"] >= 100]
# Only keep those really negative
threshold = -0.05
most_agg = most_agg[most_agg["compound"] <= threshold].sort_values("year")

In [37]:
# Display table
most_agg.sort_values(["year", "compound"])

compound  quotes_count
year speaker                  parties                           
2015 elijah cummings          Democrat   -0.339667           201
2016 elijah cummings          Democrat   -0.188794           132
     donald trump             Republican -0.052753          4094
2017 barbara lee              Democrat   -0.232446           104
     maxine waters            Democrat   -0.114713           180
     jeff flake               Republican -0.111487           133
     cory booker              Democrat   -0.078286           132
     ted lieu                 Democrat   -0.077877           100
     kirsten gillibrand       Democrat   -0.074496           156
     jeanne shaheen           Democrat   -0.056756           100
2018 maxine waters            Democrat   -0.122643           164
     kirsten gillibrand       Democrat   -0.120604           140
     mazie hirono             Democrat   -0.119052           102
     jerrold nadler           Democrat   -0.116621           131
     dianne feinstein         Democrat   -0.083949           318
     orrin hatch              Republican -0.075000           137
     richard blumenthal       Democrat   -0.072185           227
     ted lieu                 Democrat   -0.061465           109
     adam smith               Democrat   -0.054608           106
2019 xavier becerra           Democrat   -0.173166           116
     john ratcliffe           Republican -0.129743           117
     tulsi gabbard            Democrat   -0.126019           205
     mo brooks                Republican -0.120918           171
     elise stefanik           Republican -0.097369           157
     alexandria ocasio-cortez Democrat   -0.082365           279
     donald trump             Republican -0.080294           987
     devin nunes              Republican -0.062832           476
     president donald trump   Republican -0.057747          3823
     kamala harris            Democrat   -0.054848           169
     rashida tlaib            Democrat   -0.054251           107

In [36]:
g = pd.DataFrame(most_agg.groupby(["year", "parties"]).size()).rename({0: "count"}, axis=1)
g["proportion"] = g["count"] / g.groupby("year")["count"].sum()
g

count  proportion
year parties                      
2015 Democrat        1    1.000000
2016 Democrat        1    0.500000
     Republican      1    0.500000
2017 Democrat        6    0.857143
     Republican      1    0.142857
2018 Democrat        8    0.888889
     Republican      1    0.111111
2019 Democrat        5    0.454545
     Republican      6    0.545455

Observations:

**2015**:
- blabla

## 2. Impact of events 

### 2.1 Events dataset

No single dataset that we could found was adapted to our needs. Because of that, we decided to constitute a dataset ourselves by selecting interesting events from different sources.
- Wikipedia's "year in the USA events list" [link](https://en.wikipedia.org/wiki/2015_in_the_United_States);
- ACLED (The Armed Conflict Location & Event Data Project), which contains US events like protests and shootings. We select only the most important (i.e. with most fatalities) from this dataset [link](https://acleddata.com/).
- Most impactful events in US history as defined by the Encyclopedia Britannica [link](https://www.britannica.com/list/25-decade-defining-events-in-us-history).
- Most impactful events in US history as defined by the BBC [link](https://www.bbc.com/news/world-us-canada-16759233).
- Most impactful events in US history as defined by the Time [link](https://time.com/3889533/25-moments-changed-america/).
- Most impactful events accross generations, results of a research [link](https://www.pewresearch.org/politics/2016/12/15/americans-name-the-10-most-significant-historic-events-of-their-lifetimes/).


Final dataset handpicked from wikipedia (see `events_dataset.ipynb`). Actual analysis done by René.